In [1]:
#https://github.com/kapadias/mediumposts/blob/master/natural_language_processing/topic_modeling/notebooks/Introduction%20to%20Topic%20Modeling.ipynb

# Importing modules
import pandas as pd
import os
import numpy as np
import re


os.chdir('..')

# Read data into papers
papers = pd.read_csv('D:\Anaconda\python\scopus1.csv')

# Print head
papers.head()

,Authors,Author full names,Author(s) ID,Year,Source title,DOI,Link,Affiliations,Authors with affiliations,Abstract,Indexed Keywords,Author Keywords,Document Type,Source
0,Fitzgerald K.; Devonshire E.; Vaughan B.,"Fitzgerald, Kylie (23990834600); Devonshire, E...",23990834600; 54994531800; 16235143200,2020,Health Professions Education,10.1016/j.hpe.2020.09.001,https://www.scopus.com/inward/record.uri?eid=2...,"The Pain Management Research Institute, Sydney...","Fitzgerald K., The Pain Management Research In...",Purpose: The global burden of pain is consider...,NaN,Attitudes; Back pain; Curriculum; Education; O...,Article,Scopus
1,NaN,NaN,NaN,2020,Hematology. American Society of Hematology. Ed...,10.1182/hematology.2020Erratum1,https://www.scopus.com/inward/record.uri?eid=2...,NaN,NaN,[No abstract available],erratum,NaN,Erratum,Scopus
2,Gembris H.; Menze J.; Heye A.; Bullerjahn C.,"Gembris, Heiner (56404579500); Menze, Jonas (5...",56404579500; 57219867409; 57211967141; 5644582...,2020,Frontiers in Psychology,10.3389/fpsyg.2020.564736,https://www.scopus.com/inward/record.uri?eid=2...,"Faculty of Cultural Studies, Institute for Res...","Gembris H., Faculty of Cultural Studies, Insti...","The present study examines the prevalence, loc...",NaN,adolescents; high performers; music education;...,Article,Scopus
3,Kim S.-W.,"Kim, Seong-Woo (54399892700)",54399892700,2020,Proceedings of 2020 IEEE International Confere...,10.1109/TALE48869.2020.9368332,https://www.scopus.com/inward/record.uri?eid=2...,"Seoul National University, Graduate School of ...","Kim S.-W., Seoul National University, Graduate...",The fundamentally-required literacy in higher ...,Computer programming; Machine learning; Studen...,Artificial intelligence; Engineering literacy;...,Conference paper,Scopus
4,Benedicto J.; Echaves A.; Jurado T.; Ramos M.;...,"Benedicto, Jorge (6603612542); Echaves, Antoni...",6603612542; 57222029480; 57191157882; 57203283...,2020,Revista Espanola de Sociologia,10.22325/FES/RES.2020.78,https://www.scopus.com/inward/record.uri?eid=2...,"Universidad Nacional de Educación a Distancia,...","Benedicto J., Universidad Nacional de Educació...",Based on the results of a survey carried out i...,NaN,Crisis; Emancipation; Precariousness; Youth,Review,Scopus


In [2]:
papers.shape

(1000, 14)

In [3]:
# Remove the columns
Papers = papers.drop(columns=['Author(s) ID', 'Link', 'Author full names','DOI','Document Type','Source'], axis=1).sample(1000)

# Print out the first rows of papers
Papers.head()

,Authors,Year,Source title,Affiliations,Authors with affiliations,Abstract,Indexed Keywords,Author Keywords
469,Makedonskaja V.; Bykovskaia G.; Miroshnichenko E.,2020,E3S Web of Conferences,"Moscow Engineering Physics Institute, Kashirsk...","Makedonskaja V., Moscow Engineering Physics In...",The priority directions of the spiritual and m...,Engineering education; Educational events; Edu...,NaN
769,Sun H.; Shao Z.; Shang Y.; Chen B.; Zhao X.,2020,Journal of Image and Graphics,"College of Information Engineering, Capital No...","Sun H., College of Information Engineering, Ca...",Objective: Along with the wide usage of variou...,NaN,Cancelable face recognition; Complex matrix; F...
157,White B.; Drew M.; Gaughan J.; Phadtare S.,2020,Healthcare (Switzerland),"Department of Biomedical Sciences, Cooper Medi...","White B., Department of Biomedical Sciences, C...",Reports of adverse effects associated with pro...,NaN,Adverse effects; Biopsychosocial; Medically un...
396,Shan' H.; Min Z.; Kuankuan H.; Wenwei Y.; Zhix...,2020,Chinese Journal of Cerebrovascular Diseases,"Graduate School of Dalian Medical University, ...","Shan' H., Graduate School of Dalian Medical Un...",Objective To explore the relationship between ...,Article; basal ganglion; cerebrovascular disea...,Enlarged perivascular space; Executive functio...
722,Pînzariu A.I.,2020,Review of Applied Socio-Economic Research,"University of Bucharest, Romania","Pînzariu A.I., University of Bucharest, Romania",Barnett R. (2000) argues that educational inst...,NaN,Artificial Intelligence; Digitalization; Hybri...


In [4]:
Papers.isnull().sum()

Authors                        7
Year                           0
Source title                   0
Affiliations                  12
Authors with affiliations     12
Abstract                       0
Indexed Keywords             404
Author Keywords              202
dtype: int64

In [5]:
import gensim
from gensim.utils import simple_preprocess
import nltk
nltk.download('stopwords')
from nltk.corpus import stopwords

stop_words = stopwords.words('english')
print(stop_words)

['i', 'me', 'my', 'myself', 'we', 'our', 'ours', 'ourselves', 'you', "you're", "you've", "you'll", "you'd", 'your', 'yours', 'yourself', 'yourselves', 'he', 'him', 'his', 'himself', 'she', "she's", 'her', 'hers', 'herself', 'it', "it's", 'its', 'itself', 'they', 'them', 'their', 'theirs', 'themselves', 'what', 'which', 'who', 'whom', 'this', 'that', "that'll", 'these', 'those', 'am', 'is', 'are', 'was', 'were', 'be', 'been', 'being', 'have', 'has', 'had', 'having', 'do', 'does', 'did', 'doing', 'a', 'an', 'the', 'and', 'but', 'if', 'or', 'because', 'as', 'until', 'while', 'of', 'at', 'by', 'for', 'with', 'about', 'against', 'between', 'into', 'through', 'during', 'before', 'after', 'above', 'below', 'to', 'from', 'up', 'down', 'in', 'out', 'on', 'off', 'over', 'under', 'again', 'further', 'then', 'once', 'here', 'there', 'when', 'where', 'why', 'how', 'all', 'any', 'both', 'each', 'few', 'more', 'most', 'other', 'some', 'such', 'no', 'nor', 'not', 'only', 'own', 'same', 'so', 'than', '

[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\admin\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [6]:
stop_words.extend(['i', 'me', 'my', 'myself', 'we', 'our', 'ours', 'ourselves', 'you', "you're", "you've", "you'll", "you'd", 'your', 'yours', 'yourself', 'yourselves',
                   'he', 'him', 'his', 'himself', 'she', "she's", 'her', 'hers', 'herself', 'it', "it's", 'its', 'itself', 'they', 'them', 'their', 'theirs', 'themselves', 
                   'what', 'which', 'who', 'whom', 'this', 'that', "that'll", 'these', 'those', 'am', 'is', 'are', 'was', 'were', 'be', 'been', 'being', 'have', 'has', 'had', 
                   'having', 'do', 'does', 'did', 'doing', 'a', 'an', 'the', 'and', 'but', 'if', 'or', 'because', 'as', 'until', 'while', 'of', 'at', 'by', 'for', 'with', 'about', 'against', 'between', 
                   'into', 'through', 'during', 'before', 'after', 'above', 
                   'below', 'to', 'from', 'up', 'down', 'in', 'out', 'on', 'off', 'over', 'under', 'again', 'further', 'then', 'once', 'here', 'there', 'when', 'where', 'why', 'how', 'all', 'any', 'both', 'each', 'few', 'more', 'most', 'other', 'some', 'such', 
                   'no', 'nor', 'not', 'only', 'own', 'same', 'so', 'than', 'too', 'very', 's', 't', 'can', 'will', 'just', 'don', "don't", 'should', "should've", 'now', 'd', 'll', 'm', 'o', 're', 've', 'y', 'ain', 'aren', "aren't", 'couldn', "couldn't", 'didn', 
                   "didn't", 'doesn', "doesn't", 'hadn', "hadn't", 'hasn', "hasn't", 'haven', "haven't", 'isn', "isn't", 'ma', 'mightn', "mightn't", 'mustn', "mustn't", 
                   'needn', "needn't", 'shan', "shan't", 'shouldn', "shouldn't", 'wasn', "wasn't", 'weren', "weren't", 'won', "won't", 'wouldn', "wouldn't"])

def sent_to_words(sentences):
    for sentence in sentences:
        # deacc=True removes punctuations
        yield(gensim.utils.simple_preprocess(str(sentence), deacc=True))

def remove_stopwords(texts):
    return [[word for word in simple_preprocess(str(doc)) 
             if word not in stop_words] for doc in texts]


data = Papers.Abstract.values.tolist()
data_words = list(sent_to_words(data))

# remove stop words
data_words = remove_stopwords(data_words)

print(data_words[:1][0][:30])

['priority', 'directions', 'spiritual', 'moral', 'components', 'personality', 'formation', 'characterized', 'article', 'effectiveness', 'program', 'conducting', 'educational', 'events', 'different', 'levels', 'higher', 'education', 'students', 'participation', 'investigated', 'article', 'proved', 'harmonious', 'system', 'educational', 'work', 'formed', 'universities', 'russia']


In [7]:
import gensim.corpora as corpora

# Create Dictionary
id2word = corpora.Dictionary(data_words)

# Create Corpus
texts = data_words

# Term Document Frequency
corpus = [id2word.doc2bow(text) for text in texts]

# View
print(corpus[:1][0][:30])

[(0, 1), (1, 1), (2, 3), (3, 1), (4, 1), (5, 1), (6, 1), (7, 1), (8, 1), (9, 1), (10, 1), (11, 2), (12, 1), (13, 1), (14, 1), (15, 3), (16, 1), (17, 1), (18, 3), (19, 1), (20, 1), (21, 1), (22, 1), (23, 1), (24, 1), (25, 1), (26, 1), (27, 1), (28, 1), (29, 1)]


In [8]:
#Model traning
from pprint import pprint

# number of topics
num_topics = 20

# Build LDA model
lda_model = gensim.models.LdaMulticore(corpus=corpus,
                                       id2word=id2word,
                                       num_topics=num_topics)

# Print the Keyword in the 20 topics
pprint(lda_model.print_topics())
doc_lda = lda_model[corpus]

[(0,
  '0.009*"education" + 0.009*"study" + 0.006*"research" + 0.006*"health" + '
  '0.005*"students" + 0.005*"higher" + 0.004*"data" + 0.004*"among" + '
  '0.004*"teachers" + 0.004*"ci"'),
 (1,
  '0.011*"education" + 0.009*"study" + 0.006*"online" + 0.006*"students" + '
  '0.005*"higher" + 0.004*"results" + 0.004*"children" + 0.004*"language" + '
  '0.004*"used" + 0.004*"high"'),
 (2,
  '0.017*"students" + 0.012*"education" + 0.007*"study" + 0.006*"educational" '
  '+ 0.006*"higher" + 0.006*"based" + 0.005*"research" + 0.005*"learning" + '
  '0.005*"results" + 0.004*"development"'),
 (3,
  '0.009*"education" + 0.008*"data" + 0.007*"research" + 0.006*"study" + '
  '0.005*"high" + 0.005*"students" + 0.004*"results" + 0.004*"used" + '
  '0.004*"social" + 0.004*"analysis"'),
 (4,
  '0.008*"education" + 0.007*"study" + 0.007*"students" + 0.006*"online" + '
  '0.006*"higher" + 0.005*"results" + 0.005*"development" + 0.004*"system" + '
  '0.004*"level" + 0.004*"learning"'),
 (5,
  '0.009*"ed

In [9]:
import pyLDAvis.gensim as gensimvis
import pickle 
import pyLDAvis

# Visualize the topics
pyLDAvis.enable_notebook()

LDAvis_data_filepath = os.path.join('D:\Anaconda\python'+str(num_topics))

# # this is a bit time consuming - make the if statement True
# # if you want to execute visualization prep yourself
if 1 == 1:
    LDAvis_prepared = pyLDAvis.gensim.prepare(lda_model, corpus, id2word)
    with open(LDAvis_data_filepath, 'wb') as f:
        pickle.dump(LDAvis_prepared, f)

# load the pre-prepared pyLDAvis data from disk
with open(LDAvis_data_filepath, 'rb') as f:
    LDAvis_prepared = pickle.load(f)

pyLDAvis.save_html(LDAvis_prepared, 'D:\Anaconda\python'+ str(num_topics) +'.html')

LDAvis_prepared

PreparedData(topic_coordinates=              x         y  topics  cluster      Freq
topic                                               
15    -0.009193  0.000679       1        1  9.742159
7     -0.019431 -0.018985       2        1  9.342689
14    -0.003236 -0.022665       3        1  7.632015
19    -0.013882 -0.005008       4        1  7.126854
17    -0.012565 -0.009952       5        1  6.922457
10     0.002331 -0.014859       6        1  6.589847
2     -0.014704 -0.010838       7        1  6.202836
11    -0.010597 -0.016513       8        1  6.147514
6     -0.023247  0.001993       9        1  4.922846
12    -0.017847 -0.000481      10        1  4.590667
18    -0.020362  0.020746      11        1  4.565094
8     -0.016030 -0.001160      12        1  4.289230
16     0.051694 -0.028120      13        1  4.054597
0      0.025660 -0.033326      14        1  3.839677
1      0.034360  0.021871      15        1  3.348340
3     -0.013585  0.015952      16        1  3.002655
5      0.016958  0.000069      17        1  2.503371
9     -0.014516  0.039569      18        1  2.476810
4      0.008202  0.028532      19        1  1.681706
13     0.049991  0.032496      20        1  1.018636, topic_info=           Term         Freq        Total Category  logprob  loglift
14    education  1572.000000  1572.000000  Default  30.0000  30.0000
400       study  1232.000000  1232.000000  Default  29.0000  29.0000
1548     online   310.000000   310.000000  Default  28.0000  28.0000
67     students  1349.000000  1349.000000  Default  27.0000  27.0000
129        data   542.000000   542.000000  Default  26.0000  26.0000
...         ...          ...          ...      ...      ...      ...
489         two     3.851175   224.406668  Topic20  -5.9735   0.5216
100       based     4.112630   557.924403  Topic20  -5.9078  -0.3235
443       group     3.754834   323.758226  Topic20  -5.9989   0.1298
177        high     3.687418   566.457286  Topic20  -6.0170  -0.4478
208     methods     3.541486   342.586142  Topic20  -6.0574   0.0147

[1594 rows x 6 columns], token_table=       Topic      Freq         Term
term                               
10137     10  0.571447        aacsb
7651       5  0.546573  abandonment
1068       8  0.521102         abet
1068      10  0.173701         abet
1068      20  0.173701         abet
...      ...       ...          ...
6262      17  0.648498         yuan
6262      18  0.024942         yuan
3770      16  0.602425       zapata
5581       9  0.381115           μg
2362      18  0.630223         μnet

[6077 rows x 3 columns], R=30, lambda_step=0.01, plot_opts={'xlab': 'PC1', 'ylab': 'PC2'}, topic_order=[16, 8, 15, 20, 18, 11, 3, 12, 7, 13, 19, 9, 17, 1, 2, 4, 6, 10, 5, 14])